# FraudDetection - Environment Setup

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/obeshor/FraudDetection/raw/main/00_environment_setup.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github.com/obeshor/FraudDetection/blob/main/00_environment_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/github.com/obeshor/FraudDetection/blob/main/00_environment_setup.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

Before you run this notebook, make sure that you have completed the steps in [README](README.md).

In this notebook, you will setup your environment for Fraudfinder to be used in subsequent labs.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)
- [Google Cloud Storage](https://cloud.google.com/storage)
- [Pub/Sub](https://cloud.google.com/pubsub/)

Steps performed in this notebook:

- Setup your environment.
- Load historical bank transactions into BigQuery.
- Read data from BigQuery tables.
- Read data from Pub/Sub topics, which contain a live stream of new transactions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Pub/Sub
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Pub/Sub pricing](https://cloud.google.com/pubsub/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install additional packages

Install the following packages required to execute this notebook.

In [1]:
! pip install --upgrade -r 'requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of google-cloud-pipeline-components to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of g

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup your environment

Run the next cells to import libraries used in this notebook and configure some options.

Run the next cell to set your project ID and some of the other constants used in the lab.  

In [10]:
import random
import string
from typing import Union

import pandas as pd
from google.cloud import bigquery

# Generate unique ID to help w/ unique naming of certain pieces
ID = "".join(random.choices(string.ascii_lowercase + string.digits, k=5))

GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
REGION = "us-central1"
TRAINING_DS_SIZE = 1000

### Create a Google Cloud Storage bucket and save the config data.

Next, we will create a Google Cloud Storage bucket and will save the config data in this bucket. After the cell operation finishes, you can navigate to [Google Cloud Storage](https://console.cloud.google.com/storage/) to see the GCS bucket. 

In [2]:
config = f"""
BUCKET_NAME          = \"{BUCKET_NAME}\"
PROJECT              = \"{PROJECT_ID}\"
REGION               = \"{REGION}\"
ID                   = \"{ID}\"
FEATURESTORE_ID      = \"fraudfinder_{ID}\"
MODEL_NAME           = \"ff_model\"
ENDPOINT_NAME        = \"ff_model_endpoint\"
TRAINING_DS_SIZE     = \"{TRAINING_DS_SIZE}\"
"""

!gsutil mb -l {REGION} gs://{BUCKET_NAME}

!echo '{config}' | gsutil cp - gs://{BUCKET_NAME}/config/notebook_env.py

Creating gs://isogde-fraudfinder/...
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


### Copy the historical transaction data into BigQuery tables

Now we will copy the historical transaction data and ingest it into BigQuery tables. For this, we will need to run `copy_bigquery_data.py`.

In [3]:
!python3 scripts/copy_bigquery_data.py $BUCKET_NAME

File copied from gs://cymbal-fraudfinder/datagen/hacked_customers_history.txt 
		 to gs://isogde-fraudfinder/datagen/hacked_customers_history.txt
File copied from gs://cymbal-fraudfinder/datagen/hacked_terminals_history.txt 
		 to gs://isogde-fraudfinder/datagen/hacked_terminals_history.txt
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_profiles.csv 
		 to gs://isogde-fraudfinder/datagen/demographics/customer_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/terminal_profiles.csv 
		 to gs://isogde-fraudfinder/datagen/demographics/terminal_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_with_terminal_profiles.csv 
		 to gs://isogde-fraudfinder/datagen/demographics/customer_with_terminal_profiles.csv
BigQuery table created: `isogde`.tx.tx
BigQuery table created: `isogde`.tx.txlabels
BigQuery table created: `isogde`.demographics.customers
BigQuery table created: `isogde`.demographics.terminals
BigQuery t

### Check data in BigQuery

After ingesting our data into BigQuery, it's time to run some queries against the tables to inspect the data. You can also go to the [BigQuery console](https://console.cloud.google.com/bigquery) to see the data.

#### Initialize BigQuery SDK for Python 

Use a helper function for sending queries to BigQuery.

In [4]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

#### tx.tx
The `tx.tx` table contains the basic information about each transaction:
- `TX_ID` is a unique ID per transaction
- `TX_TS` is the timestamp of the transaction, in UTC
- `CUSTOMER_ID` is a unique 16-digit string ID per customer
- `TERMINAL_ID` is a unique 16-digit string ID per point-of-sale terminal
- `TX_AMOUNT` is the amount of money spent by the customer at a terminal, in dollars

In [5]:
run_bq_query(
    """
SELECT
  *
FROM
  tx.tx
LIMIT 5
"""
)

Finished job_id: c49d26b5-1541-4ed6-8092-98632a67d8e5


,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,3e4ef9ddc16cd5a520f93fab795210fa3e855007,2025-10-16 18:29:27+00:00,6306004586824062,00064542,37.160000000
1,5d6895421d0e828a34a8309ab4a16f8f8a92f344,2025-10-16 06:17:35+00:00,0291762451225654,00064542,5.130000000
2,89efd1fa98a5c8f4c22feb698500acb17183a187,2025-10-16 12:17:47+00:00,3067754972043832,00064542,84.390000000
3,59df60827bd9ea0b0a627e5de499a5e1d3e69383,2025-10-16 18:24:46+00:00,9038541296430573,00064542,88.720000000
4,45c2abe62242553628c819515a5b4c93d5fcea0e,2025-10-16 03:31:33+00:00,3884437987790018,00064542,88.390000000


#### tx.txlabels
The `tx.txlabels` table contains information on whether each transation was fraud or not:
- `TX_ID` is a unique ID per transaction
- `TX_FRAUD` is 1 if the transaction was fraud, and 0 if the transaction was not fraudulent

In [6]:
run_bq_query(
    """
SELECT
  *
FROM
  tx.txlabels
LIMIT 5
"""
)

Finished job_id: 6ed67245-d46c-4765-8bea-ea45bbbe94aa


,TX_ID,TX_FRAUD
0,37e10ce51162dbeb371403e91f7b976bfaec33a6,0
1,536f3365445f64f792d8de55c2efcad6c7298f0e,0
2,0d30e250b96cc631f30cd6163402cae864fdb046,0
3,380c8b21240a5313cae01dc513ba603c5396f975,0
4,91ca1c138117ee9d1fc0020eacd0f1e9a9092fbb,0


### Check live streaming transactions via public Pub/Sub topics

As part of the [README](README.md), you've created [subscriptions](https://console.cloud.google.com/cloudpubsub/subscription/) to public Pub/Sub topics, where there is a constant flow of new transactions. This means you have, in your own Google Cloud project, subscriptions to the public Pub/Sub topics. You will receive a Pub/Sub message in your subscription every time a new transaction is streamed into the Pub/Sub topic.

There are two public Pub/Sub topics where there is a constant stream of live transactions occurring.

The following Pub/Sub topics are used for transactions:
```
projects/cymbal-fraudfinder/topics/ff-tx
projects/cymbal-fraudfinder/topics/ff-txlabels
```

Note: If you haven't completed the steps in the README, please make sure that you complete them first before continuing this notebook, otherwise you may not have Pub/Sub subscriptions.

### Reading messages from the Pub/Sub topics

In [13]:
def read_from_sub(project_id, subscription_name, messages=10):
    """
    Read messages from a Pub/Sub subscription
    Args:
        project_id: project ID
        subscription_name: the name of a Pub/Sub subscription in your project
        messages: number of messages to read
    Returns:
        msg_data: list of messages in your Pub/Sub subscription as a Python dictionary
    """
    
    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(project_id, subscription_name)

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(received_message.message.data.decode("utf-8"))
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data

#### Reading from the `ff-tx-sub` subscription

Now let's read from the `ff-tx-sub` subscription. You should see some recent transactions (in UTC timezone).

In [14]:
messages_tx = read_from_sub(
    project_id=PROJECT_ID, subscription_name="ff-tx-sub", messages=2
)

messages_tx

Received and acknowledged 2 messages from projects/isogde/subscriptions/ff-tx-sub.


[{'TX_ID': '1eaedf454ba6a70a00b46ca0dea7101509c8ce53',
  'TX_TS': '2024-03-20 02:29:49',
  'CUSTOMER_ID': '0890080443812200',
  'TERMINAL_ID': '86017961',
  'TX_AMOUNT': 10.56},
 {'TX_ID': '58d6d17e6b4e6484396099d5ec50b72c17a84ac7',
  'TX_TS': '2024-03-20 02:29:57',
  'CUSTOMER_ID': '3883628224884795',
  'TERMINAL_ID': '66452946',
  'TX_AMOUNT': 47.62}]

#### Reading from the `ff-txlabels-sub` subscription

We will do the same with `ff-txlabels-sub` subscription, which receives the same stream of transactions as `ff-tx-sub`, but also contain the ground-truth label, `TX_FRAUD`, if the transaction is fraudulent (1) or not (0).

In [15]:
messages_txlabels = read_from_sub(
    project_id=PROJECT_ID, subscription_name="ff-txlabels-sub", messages=2
)

messages_txlabels

Received and acknowledged 2 messages from projects/isogde/subscriptions/ff-txlabels-sub.


[{'TX_ID': 'd88cc5319d15877b8e82717b936835bc5e473709', 'TX_FRAUD': 0},
 {'TX_ID': '279f251f883e0538b68735204d7488eead6660e2', 'TX_FRAUD': 0}]

### END

Now you can go to the next notebook `01_exploratory_data_analysis.ipynb`